In [1]:
from puff_model_v2 import puff_model
import torch
import os
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline 
from unit_conversion import convert_wind


#### Test Case 1: downward wind, two sources and one active - sensors below. Data is noisy-ish, bc i'm eyeballing sensor vals

In [2]:
data_dir = f"{os.getcwd()}/data"
# load in data
df_wind_ch4 = pd.read_csv(data_dir + "/wind_ch4.csv")

df_true_emission = pd.read_csv(data_dir + "/selected_controll_release.csv")
source_points = torch.tensor(np.load(data_dir + "/source_points.npy")) # shape=(n_source, 3)
sensor_points = torch.tensor(np.load(data_dir + "/sensor_points.npy")) # shape=(n_sensor, 3)
#col_points = np.load(data_dir + "/col_points.npy")  # shape=(n_col, 3)
df_bounds = pd.read_csv(data_dir + "/bounds.csv", dtype='float32')
df_puff_simulation = pd.read_csv(data_dir + '/df_sim_puff_20220502008_0.csv').fillna(0.)

tfinal = 5*60.
source_location = torch.tensor(source_points)

ws = df_wind_ch4['wind_speed.m/s'].to_numpy() # shape=(N_t,)
wd = df_wind_ch4['wind_direction'].to_numpy() # shape=(N_t,)
df_puff_simulation['x'], df_puff_simulation['y'] = convert_wind(ws,wd)


wind_function_x = lambda t : torch.tensor(CubicSpline(df_puff_simulation.index*60,df_puff_simulation.x)(t))
wind_function_y = lambda t : torch.tensor(CubicSpline(df_puff_simulation.index*60,df_puff_simulation.y)(t))

# sensor_names = ['N','W','SW','S','SE','E','NE','C1','NW']
sensor_names = ['N','E','SE','S','SW','W','NW','C1','NE']

num_sensors = len(sensor_names)
df_sensor = pd.DataFrame(sensor_points,columns = ['x','y','z'])
df_sensor['name'] = sensor_names
sensor_locations = df_sensor[['x','y','z']].to_numpy()
sensor_values_fn = dict()
for name in sensor_names:
    sensor_values_fn[name] = lambda t, f = CubicSpline(df_puff_simulation.index*60,df_puff_simulation[name]) : torch.tensor(f(t))
# ch4 = np.transpose(df_wind_ch4.iloc[:, 3:].to_numpy()) # shape=(N_obs, N_t)
sensor_names = df_wind_ch4.columns[3:]

C:\Users\andyh\AppData\Local\Temp\ipykernel_19884\792668092.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  source_location = torch.tensor(source_points)


#### Create observation data matrix of 100 obs across domain of 45 min


In [3]:
num_obs = 300
tbegin = 0*60
tend = 20*60
# source_location[0,2] = 2
obs_t = torch.linspace(tbegin,tend,num_obs)
obs = torch.zeros(num_obs, num_sensors)
obs += torch.rand(obs.shape)*0
for i in range(num_obs):
    for j in range(num_sensors):
        obs[i,j] = torch.tensor(sensor_values_fn[sensor_names[j]](obs_t[i]))


C:\Users\andyh\AppData\Local\Temp\ipykernel_19884\643698284.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs[i,j] = torch.tensor(sensor_values_fn[sensor_names[j]](obs_t[i]))


In [4]:

pm = puff_model(source_location,sensor_points, [2, 2, 1], wind_function_x, wind_function_y,[1,1],1)
Q_torch, p_torch, M_torch = pm.return_qp_matrices_new(obs,obs_t,spread=True)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x2700 and 300x300)

In [ ]:
def lasso(f,num_nz,bias=False):
    alpha = 1
    alpha_old = 0
    expand = True
    num = 0
    iter = 0
    while num != num_nz and abs(alpha_old - alpha) > 1e-12:
        x = f(alpha)
        if bias == True:
            x = x[:-1]
        print(iter)
        num = torch.sum((torch.abs(x)>1e-3).int())
        print(x)
        if num > num_nz:
            temp = alpha
            alpha += 10**iter if expand else abs(alpha_old - alpha)/2
            alpha_old = temp
        elif num < num_nz:
            temp = alpha
            alpha -= abs(alpha_old - alpha)/2
            alpha_old = temp
            # print(2,alpha)
        iter+=1
        if alpha < alpha_old:
            expand = False
    return alpha

In [ ]:
def bf_lasso(f,f_update,num_nz,bias=False):
    x = f(0)
    for i in range(len(x)):
        alpha = lasso(len(x- i-1))

In [ ]:
from cvxopt import matrix, solvers
solvers.options['show_progress'] = False
def constrained_lsq_solve(Q,p,G,h,bias=False):
    p = p.view(-1,1)
    Q = matrix(Q.detach().numpy().astype(np.double))
    p = matrix(p.detach().numpy().astype(np.double))
    G = matrix(G.detach().numpy().astype(np.double))
    h = matrix(h.detach().numpy().astype(np.double))
    return torch.tensor(np.array(solvers.qp(Q,p,G,h)['x'],dtype=np.float64)).float()

In [ ]:
from cvxopt import matrix, solvers
solvers.options['show_progress'] = False
sparse_coef = .275*torch.norm(p_torch,1)
bias = True
weighted = False
# sparse_coef = 0
p = p_torch.view(-1,1)
Q = Q_torch
if bias:
    M_torch = torch.cat([M_torch,torch.ones(M_torch.shape[0],1)],dim=1)
    if weighted:
        w = torch.sum(M_torch,dim=1)
        w-= w.min()
        w /= w.max()
        w = torch.diag(w) + 1
        # print(w)
        p = -2*(obs.T.reshape(1,-1)@w@M_torch).reshape(-1,1)
        Q = M_torch.T@w@M_torch
    else:
        p = -2*(obs.T.reshape(1,-1)@M_torch).reshape(-1,1)
        Q = M_torch.T@M_torch
else:
    if weighted:
        w = torch.sum(M_torch,dim=1)
        w-= w.min()
        w /= w.max()
        w = torch.diag(w) + 1
        # print(w)
        p = -2*(obs.T.reshape(1,-1)@w@M_torch).reshape(-1,1)
        Q = M_torch.T@w@M_torch


print(p.shape)
#only constrain non-negativity
G = torch.cat([-1*torch.eye(len(p))])
h = torch.cat([torch.zeros(len(p))])

def f(alpha):
    pp = p + alpha*torch.ones(len(p),1).reshape(-1,1)
    if bias:
        print(pp[-1,-1] - alpha)
        pp[-1,-1] -= alpha
    return constrained_lsq_solve(Q,pp,G,h,bias=bias)
alpha = lasso(f,1,bias=bias)
print(alpha)

In [ ]:
print(p.shape)

In [ ]:
p_torch_n = alpha*torch.ones(p.shape) +p
if bias:
    p_torch_n[-1,-1] -= alpha
print(p_torch_n.shape)

x = constrained_lsq_solve(Q,p_torch_n,G,h,bias=bias)
print(x)
filter = x.reshape(-1) < 1e-3
# p = p_torch_n.reshape(-1,1)
print(p.shape)
if bias:
    filter[-1] = False
print(filter)
G = torch.cat([G,torch.eye(len(filter))[filter]])
h = torch.cat([h.reshape(-1),torch.zeros(torch.sum(filter))])
x = constrained_lsq_solve(Q,p,G,h,bias=bias)
print(x)


In [ ]:
print(M_torch)


In [ ]:
M = M_torch
print(np.linalg.norm(np.array(M)@np.array(x) - obs.T.reshape(-1,1).detach().numpy(),2))
noise = np.random.rand(x.shape[0],1)*100-50
print(np.linalg.norm(np.array(M)@(np.array(x)+noise) - obs.T.reshape(-1,1).detach().numpy(),2))
print(np.linalg.norm(np.array(M)@(np.ones(x.shape)) - obs.T.reshape(-1,1).detach().numpy(),2))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.cm import get_cmap
from matplotlib.colors import Normalize

def plot_colored_arrows(wd, ws, origin_x=0, origin_y=0, color_by='index'):
    wd = np.array(wd)
    ws = np.array(ws)
    n = len(wd)
    assert wd.shape == ws.shape, "Angle and magnitude arrays must match"

    # Compute vector components
    theta = np.deg2rad(wd)
    u = ws * np.cos(theta)
    v = ws * np.sin(theta)

    # Starting positions
    x0 = np.full_like(u, origin_x)
    y0 = np.full_like(v, origin_y)

    # Values to color by (either index or magnitude)
    if color_by == 'index':
        color_values = np.linspace(0, 1, n)
        label = "Arrow Index"
    elif color_by == 'magnitude':
        color_values = (ws - ws.min()) / (ws.max() - ws.min() + 1e-8)
        label = "Magnitude"
    else:
        raise ValueError("color_by must be 'index' or 'magnitude'")

    cmap = get_cmap('coolwarm')  # red to blue
    norm = Normalize(vmin=0, vmax=1)

    # Create quiver plot
    fig, ax = plt.subplots(figsize=(8, 8))
    q = ax.quiver(x0, y0, u, v, color_values, cmap=cmap, norm=norm, angles='xy', scale_units='xy', scale=1)

    # Add colorbar
    cbar = fig.colorbar(q, ax=ax)
    cbar.set_label(label)

    ax.set_aspect('equal')
    ax.set_title("Colored Arrows with Colorbar")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.grid(True)

plot_colored_arrows(wd, ws)
plt.xlim(-3,3)
plt.ylim(-3,3)

In [ ]:
fig,ax = plt.subplots()
ax.scatter(source_location[:,0],source_location[:,1],color='red')
ax.scatter(sensor_points[:,0],sensor_points[:,1],color='blue')

labels = df_sensor['name']
for i, label in enumerate(labels):
    plt.annotate(label, (df_sensor.x[i], df_sensor.y[i]), textcoords="offset points", xytext=(0,5), ha='center')
for i in range(len(source_location)):
    plt.annotate(str(i+1), (source_location[i,0],source_location[i,1]), textcoords="offset points", xytext=(0,5), ha='center')

In [ ]:
print(sensor_values_fn['NW'](16))

In [ ]:
sensor_values_fn = dict()
for name in sensor_names:
    sensor_values_fn[name] = interp1d(df_puff_simulation.index*60,df_puff_simulation[name])

sensor_values_fn_1 = dict()
for name in sensor_names:
    sensor_values_fn_1[name] = lambda t, x = sensor_values_fn[name] : torch.tensor(x(t))



print(sensor_values_fn['C1'](24*60))
print(sensor_values_fn_1['C1'](24*60))
print(M_torch)

In [ ]:
fig, ax  = plt.subplots(3,3)
i=0
for i in range(num_sensors):
    ax[i//3,i%3].set_title(sensor_names[i])
    tt = obs_t
    ax[i//3,i%3].plot(obs_t,(M_torch[num_obs*i:(num_obs*(i+1)),:]@x).view(-1) ,color='red',label='predicted')
    ax[i//3,i%3].plot(obs_t,sensor_values_fn[sensor_names[i]](tt),color='blue',label='true')
    plt.legend()
    